<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/clean_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")

In [271]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas as pd


# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

In [2]:
df_matchs=get_file_content('data/df_all_matchs.csv')
df_ranking=get_file_content('data/df_all_ranking.csv')
df_betting=get_file_content('data/df_all_betting.csv')
df_fixed_table=get_file_content('data/df_player_fixed.csv')
df_tournaments=get_file_content('data/df_tournaments.csv')

<string>:49: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.


# Functions Used in the script

In [265]:
def find_unique_items(l1, l2):
    # Convert the lists to sets for efficient set operations
    set1 = set(l1)
    set2 = set(l2)

    # Calculate the items unique to each list
    unique_in_l1 = list(set1.difference(set2))
    unique_in_l2 = list(set2.difference(set1))

    return unique_in_l1, unique_in_l2

def add_one_year(date_string):
  #add one year tpo a date fprmat 2000.01.01 as a string
  return(datetime.strftime(datetime.strptime(date_string,"%Y.%m.%d")+timedelta(days=365),"%Y.%m.%d"))

from dateutil import parser

def convert_date_format(input_date):
    try:
        date_obj = parser.parse(input_date)
        formatted_date = date_obj.strftime("%Y.%m.%d")
        return formatted_date
    except ValueError:
        return "Invalid Date Format"


In [264]:
def clean_location(tournament_name):
  dict_hard_coding={"'":"","chi minh":"chi min","napoli":"naples","new york city":"new york","-":" ","queens club":"london",
                    "atp finals":"london"}
  tournament_name=tournament_name.split(',')[0].lower()
  for key in dict_hard_coding.keys():
    tournament_name=tournament_name.replace(key,dict_hard_coding[key])
  return tournament_name.split(',')[0].lower().strip()

def create_tournament_unique_id(df,tournament_name_col,tournament_date_col):
  df['unique_tournament_id']=df[tournament_name_col].apply(lambda x:clean_location(x))+'//'+df[tournament_date_col].apply(lambda x:convert_date_format(x))
  return df

# Speficic Dataset Cleaning

## All Matchs Cleaning

In [266]:
df_matchs_clean=df_matchs.copy()

#We make sure to remove duplicates
df_matchs_clean=df_matchs_clean.dropna(subset=['loser_name','winner_name'])

#some rounds do not make sense
list_rounds_to_remove=['International Jr Event','Wheelchair','Olympic Bronze','Champions Tour','3rd/4th Place Match']
df_matchs_clean=df_matchs_clean[~df_matchs_clean['round'].isin(list_rounds_to_remove)]

#make sure that we consider only data until today
df_matchs_clean=df_matchs_clean[pd.to_datetime(df_matchs_clean['tournament_date'])<=datetime.today()]

#remove games with "Bye" - means that the player did not play the game before
df_matchs_clean=df_matchs_clean[df_matchs_clean['loser_name']!='Bye']

#We can then see that we can remove games with no score
df_matchs_clean=df_matchs_clean[~df_matchs_clean['score'].isna()]

#We will remove games with games in 4 points
def is_correct_score(score):
  score_list=score.split('-')
  if score in ["(W/O)","(RET)","(DEF)"]:
    return True
  if '(RET)' in score or "DEF" in score:
    return True
  for set_ in score_list:
    if int(set_[0])<6 and int(set_[1])<6 and len(score_list)>1:
      # print('false returned',set_)
      return False
    if int(set_[0])>=6 or int(set_[1])>=6:
      return True
  return "?"

df_false=df_matchs_clean[df_matchs_clean['score'].apply(lambda x:is_correct_score(x))==False]
tournament_to_remove=df_false.drop_duplicates(['tournament_name','tournament_date'])
tournament_to_remove=tournament_to_remove['tournament_name']+'-'+tournament_to_remove['tournament_date'].values

df_matchs_clean=df_matchs_clean[~(df_matchs_clean['tournament_name']+'-'+df_matchs_clean['tournament_date']).isin(tournament_to_remove)]

#remove matchs that are duplicates (same players, tournament, score & round)
#this is because the scraping does not understand when the tournament has been canceled
#Only 10 games in tennis history have the same players, tournament, etc. We will consider that these will not bias the model
columns_to_check=['tournament_name','loser_name','winner_name','score','round']
df_matchs_clean=df_matchs_clean.sort_values('tournament_date')
df_matchs_clean = df_matchs_clean[~df_matchs_clean.duplicated(subset=columns_to_check, keep='first')]


#removing tournaments that have round robin for only a few games
rr=df_matchs_clean[df_matchs_clean['round']=='Round Robin']
remove_round_robin=((rr['tournament_name']+'-'+rr['tournament_date']).value_counts()<5)[((rr['tournament_name']+'-'+rr['tournament_date']).value_counts()<5)==True].index.values
df_matchs_clean=df_matchs_clean[~((df_matchs_clean['tournament_name']+'-'+df_matchs_clean['tournament_date']).isin(remove_round_robin))]

#Removal of players that have never lost
never_lost_players=find_unique_items(df_matchs_clean['winner_name'],df_matchs_clean['loser_name'])[0]
df_matchs_clean=df_matchs_clean[~df_matchs_clean['winner_name'].isin(never_lost_players)]

#Feature cleaning: round has to uniformized
df_matchs_clean['round']=df_matchs_clean['round'].replace({'Finals':'Final','Quarterfinals':'Quarter-Finals','Semifinals':'Semi-Finals'})

#Feature cleaning: remove capital letters
df_matchs_clean['winner_name']=df_matchs_clean['winner_name'].str.lower()
df_matchs_clean['loser_name']=df_matchs_clean['loser_name'].str.lower()

#Feature cleaning: we use the tournaments table to find the name of the tournament that are missing
df_tournaments_nan=df_tournaments[df_tournaments['tournament_name'].isna()].copy()
df_tournaments_nan['tournament_name']=df_tournaments_nan['url'].apply(lambda x:x.split('/')[3])

df_matchs_clean.loc[df_matchs_clean['tournament_name'].isna(),'tournament_name']=df_matchs_clean[df_matchs_clean['tournament_name'].isna()].merge(df_tournaments_nan,on=['tournament_date'])['tournament_name_y'].values

#remove tournaments that are played in a team competition (that biases the model)
#we consider that tournaments in a team are those for which we have many finals
df_final=df_matchs_clean[df_matchs_clean['round']=="Final"] #this needs to happen after round cleaning
columns_to_check=['tournament_name','tournament_date']
df_final = df_final[df_final.duplicated(subset=columns_to_check, keep=False)]
list_team_event=(df_final['tournament_name']+'-'+df_final['tournament_date']).unique()
df_matchs_clean=df_matchs_clean[~((df_matchs_clean['tournament_name']+'-'+df_matchs_clean['tournament_date']).isin(list_team_event))]

#Create a tournament unique id
df_matchs_clean=create_tournament_unique_id(df_matchs_clean,'tournament_name','tournament_date')

#After all this cleaning, we remove duplicates
df_matchs_clean=df_matchs_clean.drop_duplicates()


<ipython-input-266-7465f4d525a1>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matchs_clean['round']=df_matchs_clean['round'].replace({'Finals':'Final','Quarterfinals':'Quarter-Finals','Semifinals':'Semi-Finals'})
<ipython-input-266-7465f4d525a1>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matchs_clean['winner_name']=df_matchs_clean['winner_name'].str.lower()
<ipython-input-266-7465f4d525a1>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

## Ranking Cleaning

In [5]:
df_ranking_clean=df_ranking.copy()
df_ranking_clean=df_ranking_clean.drop_duplicates()

#clean the ranking, we have sometimes "65T" as a ranking this means that 2 players are ex aequo. We will simply remove the T.
df_ranking_clean['ranking']=df_ranking_clean['ranking'].astype(str)
df_ranking_clean['ranking']=df_ranking_clean['ranking'].str.replace('T','')

#cleaning of the player_name field
df_ranking_clean['player_name']=df_ranking_clean['player_name'].str.lower()
df_ranking_clean['player_name']=df_ranking_clean['player_name'].str.replace('  ',' ')

#Note:players in df_ranking not in df_matchs are those who never played an atp 250 game or more
#Note: players if df_matchs and not in df_ranking are those who never got better than 350 in the world

## Tournaments Cleaning

In [272]:
df_tournaments_clean=df_tournaments.copy()

#make sure that we consider only data until today
df_tournaments_clean=df_tournaments_clean[pd.to_datetime(df_tournaments_clean['tournament_date'])<=datetime.today()]

#remove tournaments with same name & date
df_tournaments_clean=df_tournaments_clean.drop_duplicates(['tournament_name','tournament_date'])

#remove doubles tournaments
df_tournaments_clean=df_tournaments_clean[~df_tournaments_clean['url'].str.contains('double')]

#clean tournament_name null
df_tournaments_clean.loc[df_tournaments_clean['tournament_name'].isna(),'tournament_name']=df_tournaments_clean.loc[df_tournaments_clean['tournament_name'].isna(),'url'].apply(lambda x: x.split('/')[3])

#add a few tournaments that are missing in atp
tournament_name_date_dict={'Bastad, Sweden':['2011.07.11'],
                           'Nur-Sultan, Kazakhstan':['2021.09.20','2022.09.20']}

for tournament_name in tournament_name_date_dict.keys():
  list_dates=tournament_name_date_dict[tournament_name]
  for date_str in list_dates:
    row_year_before=df_tournaments_clean[(df_tournaments_clean['tournament_name']==tournament_name)&
                                      (df_tournaments_clean['tournament_date']==date_str)]
    row_year_before['tournament_date']= add_one_year(row_year_before['tournament_date'].values[0])
    df_tournaments_clean=df_tournaments_clean.append(row_year_before)

#create a tournament_unique_id
df_tournaments_clean=create_tournament_unique_id(df_tournaments_clean,'tournament_name','tournament_date')

df_tournaments_clean=df_tournaments_clean.sort_values('tournament_date')

<ipython-input-272-093a8345ba3d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_year_before['tournament_date']= add_one_year(row_year_before['tournament_date'].values[0])
<ipython-input-272-093a8345ba3d>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tournaments_clean=df_tournaments_clean.append(row_year_before)
<ipython-input-272-093a8345ba3d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_year_befo

## Fixed Table Cleaning

In [7]:
df_fixed_table_clean=df_fixed_table.copy()

import re
import math

def extract_height(height_string):
    # Define a regular expression pattern to match the height format "xxxcm"
    pattern = r'(\d+\.\d+|\d+)cm'

    # Use re.search to find the height value in the input string
    match = re.search(pattern, height_string)

    # Check if a match was found
    if match:
        # Extract the matched value and convert it to a float
        height_value = float(match.group(1))
        return height_value
    else:
        # Return NaN (Not-a-Number) for invalid input
        return math.nan

from datetime import datetime

def extract_birthdate(date_string):
    possible_formats = ["%Y.%m.%d", "%Y-%m-%d", "%Y/%m/%d", "%Y%m%d", "%Y %m %d", "(%Y.%m.%d)", "(%Y-%m-%d)", "(%Y/%m/%d)", "(%Y%m%d)", "(%Y %m %d)"]

    for date_format in possible_formats:
        try:
            # Remove parentheses if present and then parse the date
            date_string = date_string.strip('()')
            birthdate = datetime.strptime(date_string, date_format)
            return birthdate.date()
        except ValueError:
            continue  # Try the next format

    # If none of the formats match, handle the error
    return None  # or raise an exception if preferred

df_fixed_table_clean['Favorite_Hand']=df_fixed_table['Plays'].apply(lambda x:x.split('-Handed')[0])
df_fixed_table_clean['BackHand']=df_fixed_table['Plays'].apply(lambda x:x.split(',')[1].split('Backhand')[0])
df_fixed_table_clean['Height (CM)']=df_fixed_table['height'].apply(lambda x:extract_height(x))
df_fixed_table_clean['Birthdate']=df_fixed_table_clean['Birthdate'].apply(lambda x:extract_birthdate(x))

df_fixed_table_clean=df_fixed_table_clean.drop(['Plays','height'],axis=1)

## Betting Cleaning

In [67]:
import numpy as np
df_betting_clean=df_betting.copy()

columns_no_odds=['Location','Tournament','Date','Winner','Loser']
columns_odds=[k for k in df_betting.columns if k not in columns_no_odds]
df_odds=df_betting[columns_odds]


#We remove games for which we have no odds
df_betting_clean=df_betting_clean[~(df_odds.isna().sum(axis=1)==len(columns_odds))]

#Cleaning of Player Names
df_betting_clean["Loser"]=df_betting_clean["Loser"].str.strip().str.lower()
df_betting_clean["Winner"]=df_betting_clean["Winner"].str.strip().str.lower()

#If we have AvgW and AvgL
df_betting_clean.loc[~df_betting_clean['AvgW'].isna(),'Final_odd_winner']=df_betting_clean.loc[~df_betting_clean['AvgW'].isna(),'AvgW']
df_betting_clean.loc[~df_betting_clean['AvgL'].isna(),'Final_odd_loser']=df_betting_clean.loc[~df_betting_clean['AvgL'].isna(),'AvgL']

# Create 'Final_odd_winner' and 'Final_odd_loser' based on the conditions
def calculate_final_odds(row):
  win_columns = [col for col in row.index[5:] if col.endswith('W') and not col.startswith('MaxW')]
  lose_columns = [col for col in row.index[5:] if col.endswith('L') and not col.startswith('MaxL')]
  # print(row,win_columns,row[win_columns].mean(skipna=True))

  final_odd_winner = row[win_columns].mean(skipna=True)
  final_odd_loser = row[lose_columns].mean(skipna=True)

  return final_odd_winner, final_odd_loser

df_betting_clean.loc[df_betting_clean['Final_odd_winner'].isna(),['Final_odd_winner', 'Final_odd_loser']] = df_betting_clean[df_betting_clean['Final_odd_winner'].isna()].apply(lambda row: pd.Series(calculate_final_odds(row)), axis=1).values

# Print the DataFrame with the new columns
df_betting_clean=df_betting_clean.drop(columns_odds,axis=1)

In [273]:
#Date corresponds to the game_date, but we need to round it to the first game recorded
def find_tournament_month(tournament_date):
  if int(tournament_date.split('-')[0])==12 and int(tournament_date.split('-')[2])>25:
    return str(int(tournament_date.split('-')[0])+1)+'-'+'01'
  return str(int(tournament_date.split('-')[0]))+'-'+ tournament_date.split('-')[1]

df_betting_clean['month']=df_betting_clean['Date'].apply(lambda x:find_tournament_month(x))
df_betting_clean['tournament_Date']=df_betting_clean.merge(df_betting_clean.groupby(["Location","month"]).min()['Date'].reset_index(),
                       on=['Location','month'],how="left")['Date_y'].values
df_betting_clean['tournament_Date']=df_betting_clean['tournament_Date'].apply(lambda x:convert_date_format(x))

#create a tournament_unique_id
df_betting_clean=create_tournament_unique_id(df_betting_clean,'Location','tournament_Date')

#manual mapping for master tournament & a few_time
list_master=['sydney//2001.11.12','shanghai//2002.11.11','houston//2003.11.10','houston//2004.11.15',
             'shanghai//2005.11.13','shanghai//2006.11.12','shanghai//2007.11.11','shanghai//2008.11.09']
dict_mapping_master={}
for k in list_master:
  dict_mapping_master[k]="tennis masters cup//"+k.split('//')[1]

dict_mapping_master['vienna//2003.11.12']='vienna//2003.10.06'
dict_mapping_master['miami//2017.01.02']='miami//2017.03.22'

df_betting_clean.loc[df_betting_clean['unique_tournament_id'].isin(list(dict_mapping_master.keys())),'unique_tournament_id']=df_betting_clean.loc[df_betting_clean['unique_tournament_id'].isin(list(dict_mapping_master.keys())),'unique_tournament_id'].map(dict_mapping_master)

#We look for tournaments in df_betting that have no map
list_unique_ids_tourn=df_tournaments_clean['unique_tournament_id'].unique()
list_unique_ids_bet=df_betting_clean['unique_tournament_id'].unique()
no_match=(find_unique_items(list_unique_ids_bet,list_unique_ids_tourn)[0])

#sometimes whe have a few days differences, we will consider them
def change_unique_id_no_match(unique_id,day_change):
  date_wrong=datetime.strptime(unique_id.split('//')[1],"%Y.%m.%d")
  date_minus_1=(datetime.strftime(date_wrong+timedelta(day_change),"%Y.%m.%d"))
  return unique_id.split('//')[0]+'//'+date_minus_1

dict_mapping={}
day_change_list=sorted([k for k in range(-20,20,1)],key=abs)
for k in no_match:
  for day_change in day_change_list:
    if change_unique_id_no_match(k,day_change) in list_unique_ids_tourn:
      dict_mapping[k]=change_unique_id_no_match(k,day_change)
      break

df_betting_clean.loc[(df_betting_clean['unique_tournament_id'].isin(no_match)&
                     df_betting_clean['unique_tournament_id'].isin(list(dict_mapping.keys()))),'unique_tournament_id']=df_betting_clean.loc[(df_betting_clean['unique_tournament_id'].isin(no_match)&
                                                                                                                                     df_betting_clean['unique_tournament_id'].isin(list(dict_mapping.keys()))),'unique_tournament_id'].map(dict_mapping)
list_unique_ids_bet2=df_betting_clean['unique_tournament_id'].unique()
no_match2=(find_unique_items(list_unique_ids_bet2,list_unique_ids_tourn)[0])

# String Cleaning

## Player Names

In [276]:
import re

def clean_player_full_3first(player_name):
  #player_name is given with the full name (first & last in that order)
  player_name = player_name.strip().lower()
  player_name = player_name.replace("-",' ').replace("'"," ").replace(".","").replace("(","").replace(")","")
  return player_name.split(' ')[-1]+' '+player_name.split(' ')[0][:3]

def clean_player_only_name_full(player_name):
  #only applied to df_betting
  #We will then merge with inclusion in the unique ID,
  #assuming that the 2 last names cannot be playing at the same time if they are not the same players
  player_name_ini=player_name
  player_name = player_name.strip().lower()
  player_name = player_name.replace("'"," ").replace(".","").replace("(","").replace(")","").replace(". ","")
  last_name=' '.join(player_name.split(' ')[:-1])
  return last_name


df_fixed_table_clean['player_name_clean_3first']=df_fixed_table_clean['Player Name'].apply(lambda x:clean_player_full_3first(x))
df_matchs_clean['winner_name_clean_3first']=df_matchs_clean['winner_name'].apply(lambda x:clean_player_full_3first(x))
df_matchs_clean['loser_name_clean_3first']=df_matchs_clean['loser_name'].apply(lambda x:clean_player_full_3first(x))
df_ranking_clean['player_name_clean_3first']=df_ranking_clean['player_name'].apply(lambda x:clean_player_full_3first(x))

df_betting_clean['clean_player_only_name_full']=df_betting_clean['Winner'].apply(lambda x:clean_player_only_name_full(x))
df_betting_clean['clean_player_only_name_full']=df_betting_clean['Loser'].apply(lambda x:clean_player_only_name_full(x))



Conclusion:
- we should remove all games of pablo vivero gonzalez	because the player_name is duplicated
- we should merge table match on betting AND ranking because this is the one for which we have 2 mapping keys.

## Clean Tournament Names

A tournament is defined as a tournament_name + date. These changes have been done above (df betting & df tournaments). See function create_unique_tournament_id

In [289]:
add_file_github('data/clean_datasets/df_betting_clean.csv',df_betting_clean.to_csv(index=False))
add_file_github('data/clean_datasets/df_ranking_clean.csv',df_ranking_clean.to_csv(index=False))
add_file_github('data/clean_datasets/df_fixed_table_clean.csv',df_fixed_table_clean.to_csv(index=False))
add_file_github('data/clean_datasets/df_matchs_clean.csv',df_matchs_clean.to_csv(index=False))
add_file_github('data/clean_datasets/df_tournaments_clean.csv',df_tournaments_clean.to_csv(index=False))